# 1.Setting 

In [1]:
import random
import pandas as pd
import numpy as np
import os
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18,resnet50,resnet101,resnet152
from torchvision import transforms

from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import albumentations as A
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore') 

/home/ericpark/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda:0


In [3]:
CFG = {
    'MODEL_NAME':"resnet152",
    'Agumentations':"blur, affin, dilation&erosion, bright&contrast",
    'data label added':"label 5, label 6 added",
    'IMG_HEIGHT_SIZE':64,
    'IMG_WIDTH_SIZE':224,
    'EPOCHS':100,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':128,
    'NUM_WORKERS':4, # 본인의 GPU, CPU 환경에 맞게 설정
    'SEED':41
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

# 2.Dataset

In [5]:
df = pd.read_csv('../data/train_label_6.csv')
del df['Unnamed: 0']

In [6]:
# 제공된 학습데이터 중 1글자 샘플들의 단어사전이 학습/테스트 데이터의 모든 글자를 담고 있으므로 학습 데이터로 우선 배치
df['len'] = df['label'].str.len()
train_v1 = df[df['len']==1]

qw=[sum(df['len']==i) for i in range(1,7)]
for k in range(1,7):
    print(k,"개 단어 갯수 : ",qw[k-1])

1 개 단어 갯수 :  23703
2 개 단어 갯수 :  28631
3 개 단어 갯수 :  13514
4 개 단어 갯수 :  9988
5 개 단어 갯수 :  7464
6 개 단어 갯수 :  5892


In [7]:
# 제공된 학습데이터 중 2글자 이상의 샘플들에 대해서 단어길이를 고려하여 Train (80%) / Validation (20%) 분할
df = df[df['len']>1]
train_v2, val, _, _ = train_test_split(df, df['len'], test_size=0.2, random_state=CFG['SEED'])

# 학습 데이터로 우선 배치한 1글자 샘플들과 분할된 2글자 이상의 학습 샘플을 concat하여 최종 학습 데이터로 사용
train = pd.concat([train_v1, train_v2])
print(len(train), len(val))

# 학습 데이터로부터 단어 사전(Vocabulary) 구축
train_gt = [gt for gt in train['label']]
train_gt = "".join(train_gt)
letters = sorted(list(set(list(train_gt))))
print(len(letters))

vocabulary = ["-"] + letters
print(len(vocabulary))
idx2char = {k:v for k,v in enumerate(vocabulary, start=0)}
char2idx = {v:k for k,v in idx2char.items()}

76094 13098
2349
2350


In [8]:
#-------------------------------------
# augmentation
# albumentation 에 있는 agumentation 기법들 사용 
def get_aug(p=0.5):
    return A.Compose([
        # blur
        A.OneOf([
         A.AdvancedBlur(rotate_limit=(90), p=p), 
         A.MedianBlur(blur_limit=(5),p=p),
         A.GaussianBlur(blur_limit=(5,7),p=p),

        ]),
        # noise
        A.OneOf([
         A.RandomRain(blur_value=1,rain_type='heavy',p=p), # rain type torrential
         A.GaussNoise(var_limit=(90.0,90.0),p=p),
        ]),

        # A.ElasticTransform(p=p)
        A.ElasticTransform(always_apply=False, p=p, sigma=4.35, alpha_affine=5),
        A.Sharpen(alpha=(0.7, 0.8), lightness=(0.8, 1.0), always_apply=False, p=p),
        
        # Birght and Contrast 
        A.RandomBrightnessContrast(brightness_limit=(-0.4,0.5),contrast_limit=(-0.4,0.5), p=p),

        # CLAHE
        A.CLAHE(clip_limit=4.0,tile_grid_size=(10,10),p=p),

    ])

#--------------------------------------
# dataset class 

class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True,argument_mode=False,tfms=get_aug()):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.train_mode = train_mode
        self.argument_mode = argument_mode
        self.tfms = tfms
        
        
    def __len__(self):
        return len(self.img_path_list)
    
    def __getitem__(self, index):
        image = Image.open(self.img_path_list[index]).convert('RGB')
        

        if self.argument_mode:
            image = np.array(image) 
            aug = self.tfms(image=image)
            image = aug['image']
            image = Image.fromarray(image)
            return image 
            #image = self.train_transform(image)


        if self.train_mode:
            #-------------------------------------------------------
            # augmented added by eric
            image = np.array(image) 
            aug = self.tfms(image=image)
            image = aug['image']
            image = Image.fromarray(image)
            image = self.train_transform(image)
        else:
            image = self.test_transform(image)


        if self.label_list is not None:
            text = self.label_list[index]
            return image, text
        else:
            return image    

    # Image Augmentation
    def train_transform(self, image):
        transform_ops = transforms.Compose([
            transforms.Resize((CFG['IMG_HEIGHT_SIZE'],CFG['IMG_WIDTH_SIZE'])),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        return transform_ops(image)
    
    def test_transform(self, image):
        transform_ops = transforms.Compose([
            transforms.Resize((CFG['IMG_HEIGHT_SIZE'],CFG['IMG_WIDTH_SIZE'])),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        return transform_ops(image)

In [9]:
# Dataset Load 
train_dataset = CustomDataset(train['img_path'].values, train['label'].values)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=CFG['NUM_WORKERS'])

val_dataset = CustomDataset(val['img_path'].values, val['label'].values,train_mode=False)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=CFG['NUM_WORKERS'])

In [10]:
image_batch, text_batch = iter(train_loader).next()
print(image_batch.size(), text_batch)

torch.Size([128, 3, 64, 224]) ('달다', '찌', '출신', '뉵', '시일', '뛰어오르다', '거리', '키우다', '종교적', '위기감수성', '가능해지다', '지도자', '색다르다', '아예', '빌다', '고장', '기대', '웽', '병', '표시하다', '땝', '채우다', '궁금하다만점', '흰색태우다', '죙', '열', '낭', '튕', '고등학교', '행동하다', '툽', '솜', '단골', '반', '해물', '붸', '세월', '종이', '올바르다', '그대', '원피스', '사고', '거듭', '파도', '시끄럽다', '난방', '주민', '항', '음식물', '미리', '랍', '재즈', '공연하다포크', '책', '내밀다', '불과하다', '먹다기초적', '걜', '인간적', '케첩', '종교적', '통과하다', '븀', '갈다', '걱정되다', '확신하다읽다', '도와주다논쟁', '기후', '평화', '언', '깔끔하다숙소', '찾아보다', '시끄럽다', '분리되다', '파일', '배', '두려워하다', '쌍둥이', '접시', '경험', '굳', '비닐', '기획지리산', '이내', '식히다', '벌어지다', '전망하다', '다가서다', '사이사이', '기술', '독하다', '불안', '각', '조사', '연두색', '달다', '뛰어나다잘못', '힛', '되게', '위기', '끎', '졈', '전반', '언덕', '휴지', '포도주', '자극하다', '프로', '소용없다순수', '뜰', '기성', '중단하다시야', '탑', '결정되다전반', '킬로그램', '성격장애인', '나란히', '팩', '설날불리다', '선생님', '차림', '뛰어놀다곧잘', '정말', '두', '고르다', '이십', '자격증', '풍경')


# 3.Modeling

In [24]:
# Model Define 
class RecognitionModel(nn.Module):
    def __init__(self, num_chars=len(char2idx), rnn_hidden_size=256):
        super(RecognitionModel, self).__init__()
        self.num_chars = num_chars
        self.rnn_hidden_size = rnn_hidden_size

        #---------------------------------------
        # RESNET
        # pretraine 된 resnet 152 사용 
        resnet = resnet152(pretrained=True)
        #---------------------------------------

        
        # CNN Feature Extract
        resnet_modules = list(resnet.children())[:-3]
        
        self.feature_extract = nn.Sequential(
            *resnet_modules,
            nn.Conv2d(1024, 512, kernel_size=(3,6), stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True)
        )

        self.linear1 = nn.Linear(2048, rnn_hidden_size)

        # RNN
        self.rnn = nn.RNN(input_size=rnn_hidden_size, 
                            hidden_size=rnn_hidden_size,
                            bidirectional=True, 
                            batch_first=True)
        self.linear2 = nn.Linear(self.rnn_hidden_size*2, num_chars)
        
        
    def forward(self, x):
        # CNN
        x = self.feature_extract(x) # [batch_size, channels, height, width]
        x = x.permute(0, 3, 1, 2) # [batch_size, width, channels, height]
         

        # resnet 50 #=============================== x shape  torch.Size([256, 11, 2048])
        batch_size = x.size(0)
        T = x.size(1)

        x = x.view(batch_size, T, -1) # [batch_size, T==width, num_features==channels*height]
        x = self.linear1(x)

        # RNN
        x, hidden = self.rnn(x)
        
        output = self.linear2(x)
        output = output.permute(1, 0, 2) # [T==10, batch_size, num_classes==num_features]
        
        return output

# 4.Loss

In [12]:
# Define CTC Loss 
criterion = nn.CTCLoss(blank=0) # idx 0 : '-'

In [13]:
def encode_text_batch(text_batch):
    text_batch_targets_lens = [len(text) for text in text_batch]
    text_batch_targets_lens = torch.IntTensor(text_batch_targets_lens)
    
    text_batch_concat = "".join(text_batch)
    text_batch_targets = [char2idx[c] for c in text_batch_concat]
    text_batch_targets = torch.IntTensor(text_batch_targets)
    
    return text_batch_targets, text_batch_targets_lens

In [14]:
def compute_loss(text_batch, text_batch_logits):
    """
    text_batch: list of strings of length equal to batch size
    text_batch_logits: Tensor of size([T, batch_size, num_classes])
    """
    text_batch_logps = F.log_softmax(text_batch_logits, 2) # [T, batch_size, num_classes]  
    text_batch_logps_lens = torch.full(size=(text_batch_logps.size(1),), 
                                       fill_value=text_batch_logps.size(0), 
                                       dtype=torch.int32).to(device) # [batch_size] 

    text_batch_targets, text_batch_targets_lens = encode_text_batch(text_batch)
    loss = criterion(text_batch_logps, text_batch_targets, text_batch_logps_lens, text_batch_targets_lens)

    return loss

# 5.Train Function

In [15]:
def train(model, optimizer,scheduler, device):
    model.to(device)
    
    best_loss = 999999
    best_model = None

    #submit = pd.read_csv('/home/eric/0.Data/0.dacon_ocr/sample_submission.csv')
    submit = {'label_gt':[],'label_pred':[]}
    submit = pd.DataFrame(submit)


    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []


        #---------------------------------------
        # df 
        df = pd.read_csv('../data/train_label_6.csv')
        del df['Unnamed: 0']
        df['len'] = df['label'].str.len()
        train_v1 = df[df['len']==1]
        df = df[df['len']>1]

        # train test split stratify
        train_v2, val, _, _ = train_test_split(df, df['len'], test_size=0.2, stratify=df['len'])
        train = pd.concat([train_v1, train_v2])
        train_gt = [gt for gt in train['label']]
        train_gt = "".join(train_gt)
        letters = sorted(list(set(list(train_gt))))
        vocabulary = ["-"] + letters
        idx2char = {k:v for k,v in enumerate(vocabulary, start=0)}
        char2idx = {v:k for k,v in idx2char.items()}

        # dataset & loader 
        train_dataset = CustomDataset(train['img_path'].values, train['label'].values,train_mode=True)
        train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=CFG['NUM_WORKERS'])

        val_dataset = CustomDataset(val['img_path'].values, val['label'].values,train_mode=False)
        val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=CFG['NUM_WORKERS'])


        for image_batch, text_batch in tqdm(iter(train_loader)):
            image_batch = image_batch.to(device)
            
            optimizer.zero_grad()
            
            text_batch_logits = model(image_batch)
            loss = compute_loss(text_batch, text_batch_logits)
        
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        _train_loss = np.mean(train_loss)

        print("# Validation")
        _val_loss,_val_labels,_val_preds = validation(model, val_loader, device)

        print("_val_labels",len(_val_labels))
        #print("label_pred",_val_preds)

        submit['label_gt']   = _val_labels
        submit['label_pred'] = _val_preds
        submit['label_pred'] = submit['label_pred'].apply(correct_prediction)
        
        #submit.to_csv(f"/home/eric/1.Source/00.Ocr/0.baseline_cnn_rnn/tmp/train_exp/sharpen_v1/result_{epoch}.csv",index=False) 
        
        #--------------------------------------------------
        # Accuracy Metric by eric 
        # baseline 에 평가메트릭이 없으므로, 자체적으로 Accuracy 메트릭 코드 작성
        # 
        az = submit

        acc_correct = 0
        acc_wrong = 0
        predict_length_wrong=0

        D = {'1':0,'2':0,'3':0,'4':0,'5':0,'6':0}
        Q = {'1':[0,0],'2':[0,0],'3':[0,0],'4':[0,0],'5':[0,0],'6':[0,0]}
        for gt, pred in zip(az['label_gt'],az['label_pred']):
            #print("gt", [i for i in gt])
            #print("pred", [z for z in pred])
            
            try:
                E = [i for i in gt]
                W =  [z for z in pred]

                for i in range(len(E)):
                    Q[str(len(E))][1] +=1
                    if E[i] == W[i]:
                        acc_correct +=1
                        Q[str(len(E))][0] +=1
                    else:
                        acc_wrong +=1
                        
            except:
                #print("predict_length_wrong case!!")
                predict_length_wrong+=1
                D[str(len([i for i in gt]))] +=1 

        print("#---------------------------------")
        print("total accuracy : ", acc_correct/(acc_correct + acc_wrong))
        #wandb.log({"total accuracy" : acc_correct/(acc_correct + acc_wrong)})

        print("accuracy by word len : ", Q )

        for i in range(2,7):
            print("accuracy by word len percent : ", f"{i} : ", Q[str(i)][0]/Q[str(i)][1])
            #wandb.log({"accuracy by word len percent {i}" :  Q[str(i)][0]/Q[str(i)][1]})
            # 1자리 수 문자는 train 으로 다 들어가므로 validation 에 들어가지 않음 
        print("pred length wrong case : ",predict_length_wrong)
        print("total case : ",len(az['label_gt']))
        print("Wrong cases by len : ",D)

        print(f'Epoch : [{epoch}] Train CTC Loss : [{_train_loss:.5f}] Val CTC Loss : [{_val_loss:.5f}]')
        #print("Validation labels : ",submit['label_gt']  )
        #print("Validation preds : ", submit['label'])

        #wandb.log({"Train CTC Loss :" : _train_loss,"Val CTC Loss : " : _val_loss }, step=epoch)

        if scheduler is not None:
            scheduler.step(_val_loss)
        
        if best_loss > _val_loss:
            best_loss = _val_loss
            best_model = model
            print("model save ########### ")
            torch.save(best_model.state_dict(), f"../exp/0.resnet152_rnn_AUGCombo_DataAdd_STR_Distortion_v1_{epoch}.pth")
    
    return best_model

# 6.Accuracy_Metric

In [16]:
# #--------------------------------------
# # Accuracy Metric by eric 
# 위의 Accuracy Metric 을 검증하기 위한 코드 
# #--------------------------------------


# az = pd.read_csv("/home/eric/1.Source/00.Ocr/0.baseline_cnn_rnn/tmp/val_gt_pred_2.csv")

# acc_correct = 0
# acc_wrong = 0
# predict_length_wrong=0

# D = {'1':0,'2':0,'3':0,'4':0,'5':0,'6':0}
# Q = {'1':[0,0],'2':[0,0],'3':[0,0],'4':[0,0],'5':[0,0],'6':[0,0]}
# for gt, pred in zip(az['label_gt'],az['label_pred']):
#     #print("gt", [i for i in gt])
#     #print("pred", [z for z in pred])
    
#     try:
#         E = [i for i in gt]
#         W =  [z for z in pred]

#         for i in range(len(E)):
#             Q[str(len(E))][1] +=1
#             if E[i] == W[i]:
#                 acc_correct +=1
#                 Q[str(len(E))][0] +=1
#             else:
#                 acc_wrong +=1
                
#     except:
#         #print("predict_length_wrong case!!")
#         predict_length_wrong+=1
#         D[str(len([i for i in gt]))] +=1 

# print("#---------------------------------")
# print("total accuracy : ", acc_correct/(acc_correct + acc_wrong))

# print("accuracy by word len : ", Q )

# for i in range(2,7):
#     print("accuracy by word len percent : ", f"{i} : ", Q[str(i)][0]/Q[str(i)][1])
#     # 1자리 수 문자는 train 으로 다 들어가므로 validation 에 들어가지 않음 
# print("pred length wrong case : ",predict_length_wrong)
# print("total case : ",len(az['label_gt']))
# print("Wrong cases by len : ",D)
            
            

# 7.Utils_Inference

In [17]:
# Submission
# 대회 기본 제공 inference 후처리 코드와 동일  
# 샘플 별 추론결과를 독립적으로 후처리
def remove_duplicates(text):
    if len(text) > 1:
        letters = [text[0]] + [letter for idx, letter in enumerate(text[1:], start=1) if text[idx] != text[idx-1]]
    elif len(text) == 1:
        letters = [text[0]]
    else:
        return ""
    return "".join(letters)

def correct_prediction(word):
    parts = word.split("-")
    parts = [remove_duplicates(part) for part in parts]
    corrected_word = "".join(parts)
    return corrected_word

def decode_predictions(text_batch_logits):
    text_batch_tokens = F.softmax(text_batch_logits, 2).argmax(2) # [T, batch_size]
    text_batch_tokens = text_batch_tokens.numpy().T # [batch_size, T]

    text_batch_tokens_new = []
    for text_tokens in text_batch_tokens:
        text = [idx2char[idx] for idx in text_tokens]
        text = "".join(text)
        text_batch_tokens_new.append(text)

    return text_batch_tokens_new
    
def validation(model, val_loader, device):
    model.eval()
    val_loss = []

    val_labels = []
    val_preds = []
    
    with torch.no_grad():
        for image_batch, text_batch in tqdm(iter(val_loader)):
            image_batch = image_batch.to(device)
            
            text_batch_logits = model(image_batch)
            loss = compute_loss(text_batch, text_batch_logits)
            #--------------------------------
            # eric 
            text_batch_pred = decode_predictions(text_batch_logits.cpu())
            val_preds.extend(text_batch_pred)
            val_labels.extend(text_batch)
            val_loss.append(loss.item())
    
    _val_loss = np.mean(val_loss)
    return _val_loss,val_labels,val_preds

# 8.Train

In [18]:
model = RecognitionModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, scheduler, device)

  8%|▊         | 50/595 [00:08<01:34,  5.74it/s]


KeyboardInterrupt: 

# 9.Inference

In [19]:
# predict 

test = pd.read_csv('../data/test.csv')

bl = []
for i,row in test.iterrows():
    qw = row['img_path']
    a = qw.replace("./test/", "../data/test/")
    bl.append(a)

test['img_path'] = bl
print(test)

               id                     img_path
0      TEST_00000  ../data/test/TEST_00000.png
1      TEST_00001  ../data/test/TEST_00001.png
2      TEST_00002  ../data/test/TEST_00002.png
3      TEST_00003  ../data/test/TEST_00003.png
4      TEST_00004  ../data/test/TEST_00004.png
...           ...                          ...
74116  TEST_74116  ../data/test/TEST_74116.png
74117  TEST_74117  ../data/test/TEST_74117.png
74118  TEST_74118  ../data/test/TEST_74118.png
74119  TEST_74119  ../data/test/TEST_74119.png
74120  TEST_74120  ../data/test/TEST_74120.png

[74121 rows x 2 columns]


In [20]:
test_dataset = CustomDataset(test['img_path'].values, None,train_mode=False)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=CFG['NUM_WORKERS'])

In [21]:
def decode_predictions(text_batch_logits):
    text_batch_tokens = F.softmax(text_batch_logits, 2).argmax(2) # [T, batch_size]
    text_batch_tokens = text_batch_tokens.numpy().T # [batch_size, T]

    text_batch_tokens_new = []
    for text_tokens in text_batch_tokens:
        text = [idx2char[idx] for idx in text_tokens]
        text = "".join(text)
        text_batch_tokens_new.append(text)

    return text_batch_tokens_new

def inference(model, test_loader, device,load=False):
    if load:
    
        model_path = "../06.Model_Weights/0.resnet152_last_v1_70.pth"
        model.load_state_dict(torch.load(model_path,map_location='cuda:0'))
        print("### model loaded ###")
        print("model : ",model_path)
    
    model.eval()
    model.to(device)
    preds = []
    with torch.no_grad():
        for image_batch in tqdm(iter(test_loader)):
            image_batch = image_batch.to(device)
            
            text_batch_logits = model(image_batch)
            
            text_batch_pred = decode_predictions(text_batch_logits.cpu())
            
            preds.extend(text_batch_pred)
    return preds

In [22]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [25]:
model = RecognitionModel()
predictions = inference(model,test_loader, device,load=True)

### model loaded ###
model :  ../06.Model_Weights/0.resnet152_last_v1_70.pth


  3%|▎         | 19/580 [00:01<00:56,  9.97it/s]


KeyboardInterrupt: 

In [26]:
# Submission 
# 샘플 별 추론결과를 독립적으로 후처리
def remove_duplicates(text):
    if len(text) > 1:
        letters = [text[0]] + [letter for idx, letter in enumerate(text[1:], start=1) if text[idx] != text[idx-1]]
    elif len(text) == 1:
        letters = [text[0]]
    else:
        return ""
    return "".join(letters)

def correct_prediction(word):
    parts = word.split("-")
    parts = [remove_duplicates(part) for part in parts]
    corrected_word = "".join(parts)
    return corrected_word

In [27]:
submit = pd.read_csv('../data/sample_submission.csv')
submit['label'] = predictions
submit['label'] = submit['label'].apply(correct_prediction)

NameError: name 'predictions' is not defined

In [ ]:
submit.to_csv('../07.Submission_Files/test_[ensemble][2]resnet152_DataAdd_STR_last.csv', index=False)

In [ ]:
submit[0:20]

,id,label
0,TEST_00000,남말
1,TEST_00001,상활
2,TEST_00002,받아들이다
3,TEST_00003,바구니
4,TEST_00004,살
5,TEST_00005,빼놓다
6,TEST_00006,인식하다
7,TEST_00007,센터
8,TEST_00008,소풍
9,TEST_00009,광주
